In [1]:
import os
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
from src.model.lit_module import LitModule
from src.data.dataset import VideoLabelDataset
import src.constants as const
from torch.utils.data import DataLoader
from src.data.dataset import (VideoLabelDataset,
                              VideoFolderPathToTensor,
                              VideoResize)

In [5]:
ll lightning_logs/version_6/checkpoints/

total 131500
-rw-rw-r-- 1 ubuntu 134652580 Jan  4 06:53 'epoch=25-step=207.ckpt'


In [9]:
checkpoint_path = './lightning_logs/version_6/checkpoints/epoch=25-step=207.ckpt'

In [10]:
model = LitModule.load_from_checkpoint(checkpoint_path)

In [11]:
dataset = VideoLabelDataset(
            const.LABELS_TABLE_QA_PATH,
            img_transform=torchvision.transforms.Compose([
                VideoFolderPathToTensor(),
                VideoResize(const.IMG_SIZE)]))

In [12]:
dataloader = DataLoader(dataset, batch_size=10, num_workers=2)

In [13]:
videos, questions, answers, hidden_states, _  = iter(dataloader).next()

In [14]:
predictions = model.eval()(videos)
predictions

tensor([[0.2236, 0.4662, 0.9885],
        [0.2244, 0.4721, 0.2638],
        [0.0537, 0.4881, 0.6125],
        [0.1221, 0.4824, 0.3305],
        [0.2819, 0.4582, 0.0720],
        [0.9013, 0.5235, 0.1032],
        [0.1281, 0.4774, 0.0551],
        [0.8868, 0.5278, 0.6848],
        [0.4278, 0.5030, 0.6455],
        [0.1881, 0.4996, 0.4343]], grad_fn=<AddmmBackward>)

In [15]:
hidden_states

tensor([[0.1563, 0.1859, 1.0000],
        [0.2144, 0.1457, 0.2500],
        [0.0180, 0.1307, 0.6667],
        [0.1042, 1.0000, 0.2917],
        [0.2625, 0.3970, 0.1250],
        [0.7976, 0.4774, 0.0417],
        [0.0381, 0.3769, 0.0833],
        [0.6894, 0.0955, 0.7500],
        [0.3908, 0.2060, 0.7083],
        [0.1503, 0.3668, 0.4167]], dtype=torch.float64)

In [18]:
mse_loss = torch.nn.MSELoss(reduction='sum')
mse_hidden = mse_loss(predictions[0:2,:].type(torch.float32),
                      hidden_states[0:2,:].type(torch.float32))
mse_hidden

tensor(0.2628, grad_fn=<MseLossBackward>)

In [33]:
((772.6472 - 543.4424)**2 + (912.8349-996.8575)**2 + (36.0000-37.2697)**2 + \
(627.9586-543.4633)**2 + (1077.1292-996.8958)**2 + (25.0000-37.2711)**2)

73323.68388475002

In [ ]:
model.eval()(videos)

In [22]:
predictions[0:2,:]

tensor([[543.4559, 996.8821,  37.2706],
        [543.4633, 996.8958,  37.2711]], grad_fn=<SliceBackward>)